# Project Scenario
<br>
A multi-national firm has hired you as a data engineer. Your job is to access and process data as per requirements.

Your boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate information made available to you as a CSV file. You should save the processed information table locally in a CSV format and as a database table. Managers from different countries will query the database table to extract the list and note the market capitalization value in their own currency.
<br><br>
## Directions

1. Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.<br><br>
2. Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.<br><br>
3. Write a function to load the transformed data frame to an output CSV file.<br><br>
4. Write a function to load the transformed data frame to an SQL database server as a table.<br><br>
5. Write a function to run queries on the database table.<br><br>
6. Run the following queries on the database table:<br>
a. Extract the information for the London office, that is Name and MC_GBP_Billion<br>
b. Extract the information for the Berlin office, that is Name and MC_EUR_Billion<br>
c. Extract the information for New Delhi office, that is Name and MC_INR_Billion<br><br>
7. Write a function to log the progress of the code.<br><br>
8. While executing the data initialization commands and function calls, maintain appropriate log entries.

## Preliminary

In [2]:
import pandas as pd
import sqlite3 as sql
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [3]:
url = "https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks"
extr_table_attr = ["Name", "MC_USD_Billion"]
table_name = "Largest_banks"
csv_path = "Largest_banks_data.csv"
exchange_rate = "exchange_rate.csv"
db_name = "Banks.db"

## Code - functions

In [4]:
# creating the extract function
# I'm going to use a different approach from the practice project, because I think that approach is somehow unnecessary 
# and stressful for nothing. Using pd.read_html() is a lot easier, but it was really fun to learn another way to do it!

def extract(url, table_attr):
    
    df = pd.read_html(url)[1]
    df = df.iloc[:, 1:3]
    df.columns = table_attr
    
    return df

In [6]:
# creating the transform function 
# We're going to add the GBP, EUR and INR currencies to the Data Frame, rounded to 2 decimal places. The exchange rates to be 
# used are located in the "exchange_rate.csv" file.

def transform(df, exch_rate_csv):
    
    exch_rate = pd.read_csv(exchange_rate)
    exch_rate = exch_rate.set_index('Currency')['Rate'].to_dict()
    
    for key, val in exch_rate.items(): # ensuring the rates to be float
        exch_rate[key] = float(val)
        
    for curr, rate in exch_rate.items():
        if curr != 'USD':
            df[f'MC_{curr}_Billion'] = round(df['MC_USD_Billion'] * rate, 2)
            
    return df

In [7]:
# creating the load functions
# Both load functions will created here, the one to load the data to a csv file and the one to load to a database file.

def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

def load_to_db(df, sql_conn, table_name):
    df.to_sql(table_name, sql_conn, if_exists= 'replace', index= False)

In [8]:
# creating the query function
# Writing a function to be able to run a SQL query in the database.

def run_query(query_stmt, sql_conn):
    print(query_stmt)
    output = pd.read_sql(query_stmt, sql_conn)
    print(output)

In [9]:
# creating the log function
# We'll be able to follow the ETL process

def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("code_log.txt", "a") as file:
        file.write(f"{timestamp}: {message}\n")

## Code - Running the project

In [10]:
log("Preliminaries complete. Initiating ETL process")

df = extract(url, extr_table_attr)

log("Data extraction complete. Initiating Transformation process")

df = transform(df, exchange_rate)

log("Data transformation complete. Initiating Loading process")

load_to_csv(df, csv_path)

log("Data saved to CSV file. Innitiating SQL connection")

conn = sql.connect(db_name)

log("SQL Connection initiated")

load_to_db(df, conn, table_name)

log("Data loaded to Database as a table, executing queries")

run_query("SELECT * FROM Largest_banks", conn)
run_query("SELECT AVG(MC_GBP_Billion) FROM Largest_banks", conn)
run_query("SELECT Name from Largest_banks LIMIT 5", conn)

conn.close()

log("Process Complete")

SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_EUR_Billion  \
0                           JPMorgan Chase          432.92          402.62   
1                          Bank of America          231.52          215.31   
2  Industrial and Commercial Bank of China          194.56          180.94   
3               Agricultural Bank of China          160.68          149.43   
4                                HDFC Bank          157.91          146.86   
5                              Wells Fargo          155.87          144.96   
6                        HSBC Holdings PLC          148.90          138.48   
7                           Morgan Stanley          140.83          130.97   
8                  China Construction Bank          139.82          130.03   
9                            Bank of China          136.81          127.23   

   MC_GBP_Billion  MC_INR_Billion  
0          346.34        35910.71  
1          185.22        19204.58  
2    